In [1]:
import pandas as pd
import numpy as np
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
from datetime import datetime, timedelta
from time import sleep
from bs4 import BeautifulSoup
from typing import Tuple, List, NamedTuple
from db_management.db import SongInfo, IDSongInfo, SongsContainer, SongsDB, DBException, ArtistInfo, AlbumInfo, SongFeatures

In [2]:
CLIENT_ID = os.environ["SPOTIFY_CLIENT_ID"]
CLIENT_SECRET = os.environ["SPOTIFY_CLIENT_SECRET"]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri='http://example.com', scope="playlist-modify-public"))

track_name = "Save Your Tears"
artist_name = "Weeknd"

#a = sp.search(q=f"track: {track_name} artist: {artist_name}", type='track', limit=1)

In [3]:
b = sp.playlist("4ZuX2YvKAlym0a8VozqV1U")

In [3]:
ENDPOINT_CHARTS = "https://www.officialcharts.com/charts/singles-chart/%s/7501/"

def generate_dates(week_gap: int = 2, years_back: int = 50):

    today = datetime.now()
    start_date = today - timedelta(days=365*years_back)
    print(len([n for n in range(int((today-start_date).days//(week_gap*7)))]))
    
    for n in range(int((today - start_date).days//(week_gap*7))):
        yield start_date + timedelta(days=n*week_gap*7)

In [4]:
def retrieve_top_songs(date: str, container: SongsContainer) -> SongsContainer:
    try:
        response = requests.get(ENDPOINT_CHARTS % date)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        items = soup.find_all("div", {"class": "chart-item"})

        for item in items:
            try:
                song = item.find("a", {"class": "chart-name"}).find("span", {"class": None})
                artist = item.find("a", {"class": "chart-artist"}).find("span", {"class": None})
    
                container.add_song(SongInfo(artist.text, song.text))
            except:
                continue
        return container
        
    except Exception as exception:
        print(exception)

In [107]:
songs = SongsContainer()
for idx, single_date in enumerate(generate_dates(week_gap=1, years_back=60)):
    date = single_date.strftime("%Y%m%d")
    retrieve_top_songs(date, songs)
    if idx % 101 == 100:
        print(idx, date)

3128
100 19660615
201 19680522
302 19700429
403 19720405
504 19740313
605 19760218
706 19780125
807 19800102
908 19811209
1009 19831116
1110 19851023
1211 19870930
1312 19890906
1413 19910814
1514 19930721
1615 19950628
1716 19970604
1817 19990512
1918 20010418
2019 20030326
2120 20050302
2221 20070207
2322 20090114
2423 20101222
2524 20121128
2625 20141105
2726 20161012
2827 20180919
2928 20200826
3029 20220803


In [108]:
len(songs)

43220

In [109]:
#with open("songs69.csv", "w", encoding="utf-8") as file:
#    file.write(songs.get_csv())

In [6]:
#songs = SongsContainer()
#songs.from_csv("songs2.csv")

In [3]:
database = SongsDB()

In [6]:
database.songs_populate_csv("songs2.csv")

In [4]:
for idx, title, artist in database.get_scraped_songs()[:5]:
    print(title, artist)

HOUSE OF THE RISING SUN THE ANIMALS
IT'S ALL OVER NOW THE ROLLING STONES
HOLD ME P J PROBY
IT'S OVER ROY ORBISON
RAMONA THE BACHELORS


In [12]:
database.close_connection()

In [7]:
def get_song_id(s: SongInfo) -> IDSongInfo:
    ext_info = sp.search(q=f"track: {s.song} artist: {s.artist}", type='track', limit=1)
    item = ext_info["tracks"]["items"][0]
    
    song_id = item["id"]
    album_id = item["album"]["id"]
    artist_id = item["artists"][0]["id"]
    title = item["name"]
    release_date = item["album"]["release_date"]
    featured = int(len(item["artists"]) > 1)
    popularity = item["popularity"]

    return IDSongInfo(song_id, album_id, artist_id, title, release_date, featured, popularity)

In [8]:
start = datetime.now()

skipped = 0

for idx, title, artist in database.get_scraped_songs()[24400:]:
    try:
        database.songs_insert(get_song_id(SongInfo(title, artist)))
        if idx % 50 == 0 and idx != 0:
            print(f"{idx}: cooldown 5s... TIME:{datetime.now()-start} SKIPPED: {skipped}")
            sleep(5)
    except DBException as exception:
        skipped += 1
        continue
    except Exception as gen_exception:
        print(gen_exception)
        sleep(10)

24450: cooldown 5s... TIME:0:00:17.342091 SKIPPED: 39
24500: cooldown 5s... TIME:0:00:40.385582 SKIPPED: 79
24650: cooldown 5s... TIME:0:01:36.293843 SKIPPED: 192
24850: cooldown 5s... TIME:0:02:49.205855 SKIPPED: 349
24950: cooldown 5s... TIME:0:03:27.405053 SKIPPED: 427
25000: cooldown 5s... TIME:0:03:50.034166 SKIPPED: 461
25100: cooldown 5s... TIME:0:04:28.863586 SKIPPED: 492
25150: cooldown 5s... TIME:0:04:52.000217 SKIPPED: 503
25200: cooldown 5s... TIME:0:05:14.106779 SKIPPED: 519
25250: cooldown 5s... TIME:0:05:37.031791 SKIPPED: 532
25300: cooldown 5s... TIME:0:05:59.165291 SKIPPED: 544
25400: cooldown 5s... TIME:0:06:40.137510 SKIPPED: 570
25450: cooldown 5s... TIME:0:07:03.065331 SKIPPED: 583
25500: cooldown 5s... TIME:0:07:28.264610 SKIPPED: 596
25550: cooldown 5s... TIME:0:07:53.641632 SKIPPED: 608
25850: cooldown 5s... TIME:0:09:48.652726 SKIPPED: 679
25950: cooldown 5s... TIME:0:10:31.560313 SKIPPED: 708
26000: cooldown 5s... TIME:0:10:55.938681 SKIPPED: 722
26050: coold

In [33]:
songs_id.save_to_csv("songs_test.csv")

In [10]:
playlists_names = [f"{year} hits" for year in range(1950, 2010, 10)]
playlists_names

['1950 hits', '1960 hits', '1970 hits', '1980 hits', '1990 hits', '2000 hits']

In [8]:
def get_playlists_songs(query: str):
    skipped = 0
    total = 0
    response = sp.search(q=query, type='playlist', limit=10)
    for it in response["playlists"]["items"]:
        name = it["name"].lower()
        if "polsk" not in name and "lat" not in name and "hity" not in name and "piosenk" not in name:
            try:
                id = it["id"]
                playlist = sp.playlist(id)
            except Exception as exception_playlist:
                print(exception_playlist)
                continue
                
            for it in playlist["tracks"]["items"]:
                total += 1
                try:
                    it = it["track"]
                    album_id = it["album"]["id"]
                    artist_id = it["artists"][0]["id"]
                    song_id = it["id"]
                    popularity = it["popularity"]
                    featured = int(len(it["artists"]) > 1)
                    title = it["name"]
                    release_date = it["album"]["release_date"]
    
                    database.songs_insert(IDSongInfo(song_id, album_id, artist_id, title, release_date, featured, popularity))
                except DBException as exception_db:
                    skipped += 1
                    continue
                except Exception as exception_general:
                    print(exception_general)
                    continue
    print(f"Added {total-skipped} out of {total}")

In [55]:
for query in playlists_names:
    get_playlists_songs(query)

Added 56 out of 640
Added 177 out of 725
Added 214 out of 533
Added 69 out of 608
Added 252 out of 700
Added 209 out of 773


In [57]:
for query in ["old english songs", "old rap", "old hiphop", "old pop", "micheal jacson", "elvis presley"]:
    get_playlists_songs(query)

Added 266 out of 641
Added 322 out of 596
Added 295 out of 660
Added 469 out of 1000
Added 79 out of 391
Added 211 out of 545


In [9]:
artists = list(map(lambda a: a[0], database.get_distinct_artists_id()))
albums = list(map(lambda a: a[0], database.get_distinct_albums_id()))
songs = list(map(lambda a: a[0], database.get_distinct_songs_id()))

In [16]:
for idx in range(len(artists)//50+1):
    batch = artists[idx*50:(idx+1)*50]
    res = sp.artists(batch)
    for artist in res["artists"]:
        try:
            id = artist["id"]
            genres = ",".join(artist["genres"])
            name = artist["name"]
            popularity = artist["popularity"]
            followers = artist["followers"]["total"]
    
            database.artists_insert(ArtistInfo(id, name, genres, popularity, followers))
        except DBException as exception_db:
            print(exception_db)

In [40]:
for idx in range(len(albums)//20+1):
    batch = albums[idx*20:(idx+1)*20]
    res = sp.albums(batch)
    for album in res["albums"]:
        try:
            id = album["id"]
            name = album["name"]
            release_date = album["release_date"]
            total_tracks = album["tracks"]["total"]
            genres = ",".join(album["genres"])
            popularity = album["popularity"]
    
            database.albums_insert(AlbumInfo(id, name, release_date, total_tracks, genres, popularity))
        except DBException as exception_db:
            print(exception_db)

        for song in album["tracks"]["items"]:
            try:
                song_id = song["id"]
                album_id = album["id"]
                artist_id = song["artists"][0]["id"]
                title = song["name"]
                release_date = album["release_date"]
                featured = int(len(song["artists"]) > 1)
                popularity = -1

                database.songs_insert(IDSongInfo(song_id, album_id, artist_id, title, release_date, featured, popularity))
                
                
            except DBException as exception_db_song:
                continue

('58RXhgjtKkp72kiSl1Eery', 'Basi musicale nello stilo dei vari artisti (instrumental karaoke tracks) Vol. 114', 'UNIQUE constraint failed: albums.album_spotify_id')


In [19]:
gg = res["albums"][0]

In [38]:
gg["tracks"]["items"][0]

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7vqyI8KIGXZwJiHwtQHpad'},
   'href': 'https://api.spotify.com/v1/artists/7vqyI8KIGXZwJiHwtQHpad',
   'id': '7vqyI8KIGXZwJiHwtQHpad',
   'name': 'Italian Hitmakers',
   'type': 'artist',
   'uri': 'spotify:artist:7vqyI8KIGXZwJiHwtQHpad'}],
 'available_markets': ['AR',
  'AU',
  'AT',
  'BE',
  'BO',
  'BR',
  'BG',
  'CA',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DK',
  'DO',
  'DE',
  'EC',
  'EE',
  'SV',
  'FI',
  'FR',
  'GR',
  'GT',
  'HN',
  'HK',
  'HU',
  'IS',
  'IE',
  'IT',
  'LV',
  'LT',
  'LU',
  'MY',
  'MT',
  'MX',
  'NL',
  'NZ',
  'NI',
  'NO',
  'PA',
  'PY',
  'PE',
  'PH',
  'PL',
  'PT',
  'SG',
  'SK',
  'ES',
  'SE',
  'CH',
  'TW',
  'TR',
  'UY',
  'GB',
  'AD',
  'LI',
  'MC',
  'ID',
  'JP',
  'TH',
  'VN',
  'RO',
  'IL',
  'ZA',
  'SA',
  'AE',
  'BH',
  'QA',
  'OM',
  'KW',
  'EG',
  'MA',
  'DZ',
  'TN',
  'LB',
  'JO',
  'PS',
  'IN',
  'BY',
  'KZ',
  'MD',
  'UA',
  'AL',


In [6]:
query = """
        SELECT song_spotify_id
        FROM songs
        WHERE popularity = -1;
        """

In [9]:
incomplete_songs = list(map(lambda s: s[0], database.get_query_database(query)))
start = datetime.now()
for idx in range(len(incomplete_songs)//50+1):
    sleep(0.5)
    if idx % 100 == 0 and idx != 0:
        print("No:", idx)
        sleep(5)
    try:
        batch = incomplete_songs[50*idx:50*(idx+1)]
        songs = sp.tracks(batch)
    except Exception as exception:
        print(exception)
        sleep(10)
        continue

    for song in songs["tracks"]:
        try:
            id = song["id"]
            popularity = song["popularity"]
            database.update_song_popularity(id, popularity)
        except DBException as exception_db:
            print(exception_db)
            continue
print(datetime.now()-start)

KeyboardInterrupt: 

In [10]:
tt = sp.audio_features(["6i6iUQMymu6N2BSjhXSHUM"])

In [11]:
tt

[{'danceability': 0.767,
  'energy': 0.907,
  'key': 11,
  'loudness': -4.537,
  'mode': 0,
  'speechiness': 0.0596,
  'acousticness': 0.117,
  'instrumentalness': 1.34e-05,
  'liveness': 0.18,
  'valence': 0.929,
  'tempo': 134.962,
  'type': 'audio_features',
  'id': '6i6iUQMymu6N2BSjhXSHUM',
  'uri': 'spotify:track:6i6iUQMymu6N2BSjhXSHUM',
  'track_href': 'https://api.spotify.com/v1/tracks/6i6iUQMymu6N2BSjhXSHUM',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6i6iUQMymu6N2BSjhXSHUM',
  'duration_ms': 197973,
  'time_signature': 4}]

In [4]:
query_2 = """SELECT song_spotify_id
             FROM songs
             WHERE song_spotify_id NOT IN (
                 SELECT song_spotify_id
                 FROM songs_features);"""

In [10]:
missing_features_songs = list(map(lambda s: s[0], database.get_query_database(query_2)))
for idx in range(len(missing_features_songs)//100+1):
    if idx % 100 == 0 and idx != 0:
        sleep(3)
        print(idx, datetime.now())
    try:
        batch = missing_features_songs[idx*100:(idx+1)*100]
        features = sp.audio_features(batch)
    except Exception as exception:
        print(exception)
        sleep(5)

    for f in features:
        try:
            database.songs_features_insert(SongFeatures(f))
        except DBException as exception_db:
            print(exception_db)
            continue
        except ValueError as exception_value:
            continue

100 2024-07-05 22:29:12.171381
200 2024-07-05 22:30:06.428267
300 2024-07-05 22:31:03.364430
400 2024-07-05 22:31:57.611360
500 2024-07-05 22:32:51.715662
600 2024-07-05 22:33:51.260527
700 2024-07-05 22:34:46.593814
800 2024-07-05 22:35:41.399333
900 2024-07-05 22:36:43.966693
1000 2024-07-05 22:37:38.208294
1100 2024-07-05 22:38:32.813399
1200 2024-07-05 22:39:30.028171
1300 2024-07-05 22:40:27.358201
1400 2024-07-05 22:41:20.620115
1500 2024-07-05 22:42:14.166757
1600 2024-07-05 22:43:09.804155
1700 2024-07-05 22:44:03.888299
1800 2024-07-05 22:44:57.095533
1900 2024-07-05 22:45:55.287452


Max Retries reached


http status: 429, code:-1 - /v1/audio-features/?ids=5FA9XC36IlV9bnhMqqzNc9,5FAAnScP4CJfEzecfnlXJI,5FAC0o51hKBJZHTL423thO,5FACElUQ6euTQx5VkWaJk9,5FAHHNzy5vDIXoA42A6fhx,5FAWodjHblif7DksmayBmq,5FAl0lIi2CUgrjMab1is7d,5FAopPQlENd6rZNNdSl8Nd,5FAvUzCQzEMmx5WA1C3DLo,5FAzAZ8E2611bVaxvNHhVN,5FB0Vxo53wb9HTkK51AicO,5FB5hFjIuwjkPqsU8Mc9AR,5FBAnvsSuOwQmxk6vRp1N4,5FBNbJHO4LeRzAUXKuOfH6,5FBOSDaBez5mCxeP6Yfj7i,5FBU818hvRs84RqDJKt0QA,5FBaWYaoA8j260ctveOa4a,5FBdc4v8QmpZ8eh170V2X4,5FBihhHcMTux7eAcKZNuQ4,5FBlH5K04d7ZjZQ8oXOyIN,5FBvGDfcEshfwMMTwR1kgm,5FBvSSi5wDo6jZV1mG3v4E,5FBxTvwsLYzBaC8PDQNXWl,5FBy3r3BVANmwXJEDyzkvF,5FC1CK9ghEPAc3zqBm5jMh,5FC7GUM0mli2O5f4OpWtUi,5FCBE7NTkbNykn2X7EzgZj,5FCFHWLwb6ANrxREbt9m1G,5FCM1Xr1C9yTKzc4FInetw,5FCk0gt3OuYWA7QNfdZiwm,5FCm7llRnvLPZMF34YQBCH,5FCsOkY2PjWvPjkGCthLYN,5FCtbCsDVx6C1q45lCd7ot,5FD0ciPSjuDqCOULQR0gNi,5FD5M13jPV02N6iL4yiA8t,5FDAbQDMHlVl223ytfd4qZ,5FDIbuZB5Hu9yAJW8DFPxD,5FDLWXaTqGoCrPIciIiLQy,5FDVnNFyPb0jQ8Ozo01Q8R,5FDjzNkpT7volw6by5SvXg,5FDn1oWUD5BXL2HCfmCk2F,5FDtw

Max Retries reached


http status: 429, code:-1 - /v1/audio-features/?ids=5FKfJK2UBtXB0c4rRF0Oc8,5FKgUis8pUFl0GjOtNqcNx,5FKkbyGm42hw1y08l2knlO,5FKnIqEQzM3wTI1nFZDGCi,5FKwhXYPXeCbYHYbl0TQLV,5FKxUCFLvJ8EPe4vkRmqR6,5FL2gq864CgYGnbYe61Yun,5FL2yYu2EXhAlfKgolhrgV,5FL7iIANnFFPFjJmgxafej,5FLDjdbddZhUlPdXPfsZaW,5FLHMNGjvwvvyk2gx5rNue,5FLRDjcoOwVb6UocUoGhvj,5FLY7KSwlT9FPGTwIBRenp,5FLctoy4mpCxLJ1QPneTOV,5FLhpX54vkWQo864zs77Ju,5FLiZgbDEqR2MNStVpbZT7,5FLrjKw2dRrX9ob2dcuFel,5FLsSmUyrw523DJKyu2658,5FLzLuPyBEppywIJprqwDz,5FM1V3qjHroqsXRBbL57rW,5FM1ZTDI8SGrYFHN1ApW89,5FM2zyvBsSQtx9HXZ8dFsc,5FM5APkms1lJOqOEq6G3N7,5FM7h8pdDyIJdJgJgOFqtD,5FMGNsThycGCsbAVwbWpah,5FMMgA0fkRE9j7DpN7yQpz,5FMT9Bw1JPrhUV95IXTgut,5FMTlViZYqdItjZ0ug0aBD,5FMXrphygZ4z3gVDHGWxgl,5FMdVXyQs9Pf62GFTkpcE0,5FMgj9WOMoDGotAvljvKLw,5FMlvzPFVvHTvz8V2bwvpp,5FMp9fcqWBCvs5pto9AZze,5FMrzyDTPhquQo53IVR1F6,5FMs8vjbznIKMuUSlQ1RtO,5FMufKoGYYQyGbzjVAN6zE,5FNF93omBMQhN1yRmGf1vN,5FNISB6H3YwBw5XdNm0JbK,5FNR7DtBHTdLCLsDF0y3kH,5FNVsJvXh3sIYW0Do5VhCN,5FNrA82MO7pD22QQ25vMZw,5FO2q

...

Max Retries reached


http status: 429, code:-1 - /v1/audio-features/?ids=5FmozEXMXttaTyrjtxuLoS,5Fmwhqt1zOi0uPYa1nl72k,5FnFUhTm3M5M8exXQxztoR,5FnLfP3KdwRzs160NZCFlK,5FnRzWYvqvQVshWubWBjCb,5FnalRiMUIcml73A8OklAO,5FndkZ2bqwMfeoxBSUVyHd,5FngBFHrHImcDy8Y42eEcC,5FnhXrjIMZU4lJ6Uh6SDrY,5FnpXVgDOk2sLT58qM22Of,5Fnw1gkVNMw6ErH9XAl8sJ,5FnzFcq29Ape7nO9bNMW8P,5Fo8DWWrCLMsiweeAI1fnn,5FoF4Qui737QPtbtzi5fI8,5FoLY81XGJKqdP5gBWJpNH,5FodabKKX86s5zWtdefaJo,5FojXlJVfhN4VOAdWHpJu7,5FosUpflq870kCMe9AqXDL,5FpI2CgNLZnvVMsS3rjtr2,5FpIuU1mjMS4er60UV5mo2,5FpMCPbmZIrVlf3zxC6eqV,5FpPOzsTOv7Z61ADbydhPR,5FpRRYLiHvFeiTFpY6lMep,5FpTW0fs2UxBNXZp2cl6gy,5FpUWJxD9evfpDj8pPDbqT,5FpanmApDmOeQ2E3H6N8AH,5FpasWdix9ko9sjSYHt5a0,5Fpbw9DCdROeT1JtJWi7lR,5Fpd23MPNCQ8xH2Q1CPhSG,5FpdDLWtWBuqngEEcq2Pg4,5Fpe2DMiC7BujUQeZFY6IV,5FpenjlldEn0NgPLSL3z3T,5FppOG2iMfnIuMY23qRUs7,5FptnjFyLyWrqo63GaXZYO,5FpvasUOApojcyIY3C0TaZ,5Fq0a4w2bJLVE3PHeacush,5FqZcSjh8TSIi0L2lnH8xt,5FqhYfbqbKyBv0ay73cmdi,5FqlDVyNf0FuHKAPS8a6K4,5Fqxl4oATW5BNpm6AsTDFh,5Fqz9XCNYMbdKwu02QNbNo,5Fr61

Max Retries reached


http status: 429, code:-1 - /v1/audio-features/?ids=5FxHVdW7neOVeA64Iha8aI,5FxILGl9q6zbAmXLSOi9db,5FxKLNGq0RLOczQ0MmTHyc,5FxL92Ty9DXcIWU0pj6vZo,5FxTU7iPXBy2xdnUr6ZEpG,5FxXx7wf5lPSTA2RSWxBwh,5Fxl4dYtAVnJDCM2FvU8U3,5FxlWCz8z4vzWUtkB0xnbr,5FxqNgIRDT64gBnRkkvwaT,5FxzqQXSF4f95rfinUWG5g,5Fy40smNxk0j5OpFMtvfaU,5Fy9TfSFwubMVyc3bNB6mr,5FySGqJJ5COyi0z0L5Fi4z,5FySQ29KDAOUXTezlE0ckt,5FyTu5E0oOFjUCMN69sn12,5FyWhCyqT96qSXHkiYNFrc,5FyXQQwmU7PbqUD2SFRyfy,5FyXW0St7lUEZc6uTSrOwY,5FyYyTIwKv4JWRPfLZ2TOS,5Fycw39lgKJdRJKOfbhayd,5FyjYmQHOwHfc2gCU8umYX,5Fz9e34Sqg2S9o5uYL5FXP,5FzDyeZ0YRCYpcRnN9uYzr,5FzNH3waRnIV4qO0XHa1WF,5FzRR8cARkKuXoC4wcCaSB,5Fzd49zJfgOt6LJh2IOmlc,5FzfJkVJSn8JrYe58uM5O5,5FzqMv43UH1dg97uIhSQ2n,5FzuDFYmUFcvjraadJXv28,5FzxdV4Ylq4xAXLqoNHY64,5G01k6KteKbAgat7pp43tC,5G03hhtEgGayM8Bnaj8DTz,5G04a8o6Ta2MzLlrH4QdQN,5G0DyOpl3fItYovtSgtwif,5G0Kue73WJvbaGiTntG5c8,5G0QYm7zA7sl5kB6IerQgg,5G0RFai2exLsIrcvqmVn8f,5G0U0rfILmGkXSVXLOEy5l,5G0kUVTirIzu9LLQRMPY0y,5G0oVoL309pqsvGDzhMOwx,5G0owsrl0JKDab09jNPLgf,5G12g

KeyboardInterrupt: 

In [12]:
sp.audio_features(batch)

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=5G6Gt9RtNaeIE418pd4yre,5G6KreV4UFfAQd8UPtt3ra,5G6Q7xpO1KP6P7HS4Ram4c,5G6QUnoweLtNuyifxzENum,5G6Ro6FdWWdOcqyfj3Ojun,5G6UuwO9HYIftl4uVhMZCW,5G6XoIbrQqBQLca2e2FxHp,5G6bHVKVzNM5SUOPomtwXt,5G6dkLyGx4RfSdHuRpwstI,5G6q1R4wIbLzkLxrG1bX2n,5G6rzSdyZhLLb92sRWOkK1,5G6x3QgKSzos6khVmDa3rI,5G6y36WN00yNZUDAbWWHTA,5G71veRZg6uFn42p4Eg9rP,5G760aZ3umyQclyaU3mxd5,5G7P83QgRGqnzJZmMJuxOH,5G7Pk8gMew5GgtFmCxAoIB,5G7RLvlWP1HBRY1D0kdcpB,5G7dXdAuz3VceUFySK9FAJ,5G7fyEZMDpiwRrXL8m56DR,5G7hpSb7x8A0MW3dl1GhmP,5G7mtIKow3zPh2zhtmbWW2,5G7uLHtoEW140QOcBpJlfz,5G81m5658cP3mPjjl7FAqF,5G83ndqN0hufipU39zWFIR,5G887SDY6YTb7Dwvu39gqI,5G8A96P0s48TwOfSsb4cZ6,5G8XLK3OBG9v2JhkaeRnKT,5G8e1Hp6jjYV4hHkaw1n3w,5G8tVyXR8Z2GXnrrSqeWdw,5G8yB40iqRUbO6NgGojn74,5G976RmOXLKXedSWAe21N6,5G9I420bFChEFKLHAHD3nd,5G9KqLxNVUYbJ10padRJgG,5G9OdOoRvKly0Jraf3O2A8,5G9Q7btIj9ZBW0OEXS5RMF,5G9QVHTR7ba3VBHQI0krfd,5G9Vy0SF7a0sBYMRFM6lWN,5G9bSbsJ79Q3N0MIJdIsCa,5G9kop64Upk14SiLlCk99V,5G9o0SKDRR